# Logistic Regression example

In [2]:
// cargo install the newest version
// this may take a while, needs to download and install the library
:dep smartcore = { git = "https://github.com/smartcorelib/smartcore", branch = "v0.5-wip" }

In [9]:
use smartcore::dataset::iris::load_dataset as iris_load;
use smartcore::dataset::breast_cancer::load_dataset as bc_load;
use smartcore::dataset::diabetes::load_dataset as db_load;
use smartcore::linear::logistic_regression::LogisticRegression;
use smartcore::linalg::basic::matrix::DenseMatrix;
// Model performance
use smartcore::metrics::accuracy;

use smartcore::model_selection::{cross_val_predict, cross_validate, KFold};

## basic prediction and accuracy

In [4]:
// Load Iris dataset
let iris_dataset = iris_load();

// Turn Iris dataset into NxM matrix
// Input data
let x: DenseMatrix<f32> = DenseMatrix::new(
    iris_dataset.num_samples,      // num rows
    iris_dataset.num_features,     // num columns
    iris_dataset.data,             // data as Vec
    false,                         // column_major
);
// These are our target class labels
let y: Vec<u32> = iris_dataset.target;

// Fit Logistic Regression to Iris dataset
let lr = LogisticRegression::fit(&x, &y, Default::default()).unwrap();
let y_hat = lr.predict(&x).unwrap(); // Predict class labels

// Calculate training error
println!("accuracy: {}", accuracy(&y, &y_hat)); // Prints 0.98

accuracy: 0.9866666666666667


## basic cross validation

In [8]:
// we need to import this as LogisticRegression is a SupervisedEstimator
// and we need its traits
use smartcore::api::SupervisedEstimator

// This example is expired by
// https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_predict.html
// Load dataset
let breast_cancer_data = bc_load();
let x: DenseMatrix<f32> = DenseMatrix::new(
    breast_cancer_data.num_samples,
    breast_cancer_data.num_features,
    breast_cancer_data.data,
    false
);
// These are our target values
let y = breast_cancer_data.target;
// cross-validated estimator
let results = cross_validate(
    LogisticRegression::new(),
    &x,
    &y,
    Default::default(),
    KFold::default().with_n_splits(3),
    &accuracy,
)
.unwrap();
println!(
    "Test score: {}, training score: {}",
    results.mean_test_score(),
    results.mean_train_score()
);


Test score: 0.9543488350505894, training score: 0.9710063417117993


## basic cross validation with prediction

In [13]:
use smartcore::linear::linear_regression::LinearRegression;

// Load Diabetes dataset
let diabetes_data = db_load();
let x = DenseMatrix::new(
    diabetes_data.num_samples,
    diabetes_data.num_features,
    diabetes_data.data,
    false
);

// These are our target values
let y = diabetes_data.target;
// Generate cross-validated estimates for each input data point
let y_hat = cross_val_predict(
    LinearRegression::new(),
    &x,
    &y,
    Default::default(),
    KFold::default().with_n_splits(10),
)
.unwrap();


println!(
    "y hat {:?}",
    y_hat
);


y hat [207, 73, 179, 162, 126, 105, 77, 121, 162, 204, 97, 101, 114, 167, 103, 181, 214, 180, 146, 127, 123, 89, 110, 259, 161, 146, 94, 184, 122, 179, 158, 70, 265, 110, 78, 92, 202, 162, 238, 137, 153, 75, 151, 80, 218, 128, 140, 110, 71, 193, 158, 166, 132, 160, 135, 68, 201, 79, 94, 137, 113, 182, 65, 99, 114, 192, 146, 125, 115, 120, 74, 233, 143, 124, 150, 127, 191, 75, 160, 87, 176, 124, 64, 153, 52, 164, 43, 152, 80, 104, 79, 189, 193, 58, 107, 124, 207, 211, 126, 137, 168, 109, 150, 160, 161, 112, 69, 158, 230, 148, 40, 120, 150, 206, 292, 190, 214, 235, 168, 147, 153, 199, 220, 174, 169, 190, 53, 105, 91, 208, 244, 64, 114, 68, 147, 236, 59, 235, 250, 248, 155, 222, 171, 121, 178, 234, 186, 229, 118, 182, 208, 144, 196, 125, 148, 197, 146, 124, 86, 233, 84, 230, 148, 195, 142, 74, 54, 260, 225, 231, 49, 86, 220, 94, 165, 119, 155, 220, 99, 166, 176, 88, 173, 161, 200, 186, 198, 63, 152, 117, 193, 123, 93, 139, 158, 170, 98, 191, 150, 179, 94, 70, 167, 199, 177, 235, 158, 214,

In [ ]:
// TODO: try to plot xy with plotter
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }

// Assemble XY dataset for the scatter plot
// let xy = DenseMatrix::from_2d_vec(
//     &y_hat
//         .into_iter()
//         .zip(y.into_iter())
//         .map(|(x1, x2)| vec![x1, x2])
//         .collect(),
// );